# Проверка показателей Открытых Данных

Обновленные проверки от 30.05.2018

Показатели:
* 001 - Количество заявок на потребительские кредиты **с 2013**
* 002 - Средняя сумма заявки на потребительский кредит **с 2013**
* 003 - Количество заявок на ипотечные кредиты **с 2013**
* 004 - Средняя сумма заявки на ипотечный кредит **с 2013**
* 007 - Количество новых депозитов **с 2014**
* 008 - Средняя сумма новго депозита **с 2014**
* 009 - Средние расходы по картам **с 2014**
* 011 - Средний чек в формате фастфуд
* 012 - Средний чек в формате ресторан
* 014 - Средние траты в формате фастфуд
* 015 - Средние траты в формате ресторан
* 020 - Средняя зарплата **с 2015**
* 021 - Средняя пенсия **с 2014**
* 038 - Рублей в среднем на текущем счете человека **с 2014**
* 039 - В среднем депозитов в рублях на человека **с 2014**

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd 
import os
import re
import random
import warnings
import datetime as dt

from transliterate import translit
from __future__ import division
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import DBSCAN
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container{width:95% !important;}</style>"))

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',160)

## Загружаем файлы

In [71]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
dateparse2 = lambda x: pd.datetime.strptime(x, '%m-%d-%Y')
dateparse3 = lambda x: pd.datetime.strptime(x, '%m.%d.%Y')
dateparse4 = lambda x: pd.datetime.strptime(x, '%d-%m-%Y')

def load_dataset(name_of_file):
    '''
    Метод загружает данные из csv в pandas dataframe
    '''
    # 2 вида записи дат существует в файлах, пробуем первый или второй
    try:
        df = pd.read_csv(name_of_file, encoding='utf-8', sep='|', header=0, dtype={'main_ind': np.float64}, decimal=',', parse_dates=['ReportDate'], date_parser=dateparse2)
    except:
        df = pd.read_csv(name_of_file, encoding='utf-8', sep='|', header=0, dtype={'main_ind': np.float64}, decimal=',', parse_dates=['ReportDate'], date_parser=dateparse2)
    #else:
    #    df = pd.read_csv(name_of_file, encoding='utf-8', sep='|', header=0, dtype={'main_ind': np.float64}, decimal=',', parse_dates=['ReportDate'], date_parser=dateparse)   
    df = df.rename(columns={df.columns[0]:'ReportDate',df.columns[1]:'subj'})
    df.ReportDate = pd.to_datetime(df.ReportDate)
    #Записываем названия регионов транслитом
    df.subj = df.subj.map(lambda x: translit(x, reversed=True) if x != 'UNKNOWN'  else x)  
    
    return df

In [77]:
PARAMETER_NUMBER = '009'
CURRENT_DATE = '2018_09_04/'
PATH_TO_ROOT_DIRECTORY = 'data/'

PATH_TO_DEV_DIRECTORY = PATH_TO_ROOT_DIRECTORY + CURRENT_DATE + 'Dev/'
PATH_TO_PUB_DIRECTORY = PATH_TO_ROOT_DIRECTORY + CURRENT_DATE + 'Pub/'
PATH_TO_SITE_DIRECTORY = PATH_TO_ROOT_DIRECTORY + CURRENT_DATE + 'Site/'

#data = load_dataset(PATH_TO_DEV_DIRECTORY + 'check_'+ PARAMETER_NUMBER +'.txt')
data = load_dataset(PATH_TO_DEV_DIRECTORY + PARAMETER_NUMBER + '_check' +'.txt')
#data = load_dataset(PATH_TO_DEV_DIRECTORY + 'check_'+ PARAMETER_NUMBER +'_v3.txt')
data.head()

,ReportDate,subj,cnt_client_tid,sum_trans_cnt,main_ind,main_ind_sum,main_ind_median,main_ind_avg,main_ind_procentile_10,main_ind_procentile_90,main_ind_max,main_ind_min,main_ind_sd,main_ind_descr,secondary1_ind_sum,secondary1_ind_median,secondary1_ind_avg,secondary1_ind_procentile_10,secondary1_ind_procentile_90,secondary1_ind_max,secondary1_ind_min,secondary1_ind_descr,secondary1_ind_sd,secondary2_ind_sum,secondary2_ind_median,secondary2_ind_avg,secondary2_ind_procentile_10,secondary2_ind_procentile_90,secondary2_ind_max,secondary2_ind_min,secondary2_ind_sd,secondary2_ind_descr,population_time
0,2014-01-15,Rossija,28995493,?,6936.2149,2.011190e+11,1600.0,6936.2149,148.8,16108.060,2.174894e+07,0.01,27384.8371,009 Средние расходы по картам.,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,06.07.2117 17:50:23
1,2014-02-15,Rossija,29460943,?,6343.7634,1.868933e+11,1572.9,6343.7634,140.0,14766.380,1.951002e+07,0.01,24783.0458,009 Средние расходы по картам.,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,06.07.2117 17:50:23
2,2014-03-15,Rossija,31014964,?,7404.6714,2.296556e+11,1884.0,7404.6714,150.0,17562.630,1.765036e+07,0.01,27560.3622,009 Средние расходы по картам.,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,06.07.2117 17:50:23
3,2014-04-15,Rossija,31361686,?,6969.3603,2.185709e+11,1769.0,6969.3603,150.0,16364.000,2.199201e+07,0.01,26518.3461,009 Средние расходы по картам.,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,06.07.2117 17:50:23
4,2014-05-15,Rossija,31858357,?,7353.1625,2.342597e+11,1901.0,7353.1625,150.0,17421.404,3.243784e+07,0.01,27588.1739,009 Средние расходы по картам.,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,06.07.2117 17:50:23


In [78]:
list_of_columns = ['ReportDate','subj','cnt_client_tid', 'sum_trans_cnt', 'main_ind','main_ind_sd',
                   'main_ind_sum','main_ind_median', 'main_ind_procentile_10', 'main_ind_procentile_90', 'main_ind_max', 'main_ind_min']

list_of_columns_short = ['ReportDate','subj', 'main_ind']

data = data[list_of_columns_short]
#data = data[list_of_columns]
data.head()

,ReportDate,subj,main_ind
0,2014-01-15,Rossija,6936.2149
1,2014-02-15,Rossija,6343.7634
2,2014-03-15,Rossija,7404.6714
3,2014-04-15,Rossija,6969.3603
4,2014-05-15,Rossija,7353.1625


In [79]:
data.ReportDate.unique()

array(['2014-01-15T00:00:00.000000000', '2014-02-15T00:00:00.000000000',
       '2014-03-15T00:00:00.000000000', '2014-04-15T00:00:00.000000000',
       '2014-05-15T00:00:00.000000000', '2014-06-15T00:00:00.000000000',
       '2014-07-15T00:00:00.000000000', '2014-08-15T00:00:00.000000000',
       '2014-09-15T00:00:00.000000000', '2014-10-15T00:00:00.000000000',
       '2014-11-15T00:00:00.000000000', '2014-12-15T00:00:00.000000000',
       '2015-01-15T00:00:00.000000000', '2015-02-15T00:00:00.000000000',
       '2015-03-15T00:00:00.000000000', '2015-04-15T00:00:00.000000000',
       '2015-05-15T00:00:00.000000000', '2015-06-15T00:00:00.000000000',
       '2015-07-15T00:00:00.000000000', '2015-08-15T00:00:00.000000000',
       '2015-09-15T00:00:00.000000000', '2015-10-15T00:00:00.000000000',
       '2015-11-15T00:00:00.000000000', '2015-12-15T00:00:00.000000000',
       '2016-01-15T00:00:00.000000000', '2016-02-15T00:00:00.000000000',
       '2016-03-15T00:00:00.000000000', '2016-04-15

In [87]:
data.subj.unique()

array(['Rossija', 'Altajskij kraj', "Amurskaja oblast'",
       "Arhangel'skaja oblast'", "Astrahanskaja oblast'",
       "Belgorodskaja oblast'", "Brjanskaja oblast'",
       "Vladimirskaja oblast'", "Volgogradskaja oblast'",
       "Vologodskaja oblast'", "Voronezhskaja oblast'",
       "Evrejskaja avtonomnaja oblast'", "Zabajkal'skij kraj",
       "Ivanovskaja oblast'", "Irkutskaja oblast'",
       'Kabardino-Balkarskaja Respublika', "Kaliningradskaja oblast'",
       "Kaluzhskaja oblast'", 'Kamchatskij kraj', "Kemerovskaja oblast'",
       "Kirovskaja oblast'", "Kostromskaja oblast'", 'Krasnodarskij kraj',
       'Krasnojarskij kraj', "Kurganskaja oblast'", "Kurskaja oblast'",
       "Leningradskaja oblast'", "Lipetskaja oblast'",
       "Magadanskaja oblast'", 'Moskva', "Moskovskaja oblast'",
       "Murmanskaja oblast'", 'Ne opredeleno',
       'Nenetskij avtonomnyj okrug', "Nizhegorodskaja oblast'",
       "Novgorodskaja oblast'", "Novosibirskaja oblast'",
       "Omskaja oblast

In [80]:
#количество месяцев и уникальных регионов
print('Количество месяцев: ', data.ReportDate.nunique())
print('Количество уникальных регионов: ', data.subj.nunique())

Количество месяцев:  54
Количество уникальных регионов:  85


In [81]:
#приводим даты к середине месяца
data.ReportDate = data.ReportDate.map(lambda x: x.replace(day=15))

## Check regions-dates

Проверка наличия всех дат и регионов в датасете: 
* Даты сравниваются со списком dates_true из диапазона *START_DATE* : *END_DATE*;
* Регионы сравниваются со списком из *subj.txt*, включающим Беларусь, Казахстан, Украину, UNKNOWN.

In [82]:
START_DATE = '2014-01-15'
END_DATE = '2018-07-15'

#генерируем эталонные даты dates_true для сравнения
dates_true = pd.date_range(START_DATE, END_DATE, freq='M')
dates_true = dates_true.map(lambda x: x.replace(day=15))

In [83]:
# Проверка присутствия регионов по дате в датасете по списку регионов в subj.txt
def check_regions(date, regions_true, regions, wrong_regions):
    # Находим разницу множеств реионов и эталонного списка регионов
    diff = list(set(regions_true).symmetric_difference(set(regions)))
    
    # 85 - нормальное число регионов
    if set(diff) != set(wrong_regions):
        print('WARNING! Date: ', date, 'Wrong Number of regions: ', len(regions))
        print('REGIONS missed in dataset: ', diff, '\n')

# Проверка присутствия дат по региону в датасете
def check_dates(region, dates_true, dates):
    # Находим разницу множеств дат и эталонного списка дат
    diff = list(set(dates_true).symmetric_difference(set(dates)))
    
    if len(diff) != 0:
        print('WARNING! Region: ', region,' Number of dates in dateset: ', len(dates))
        print('DATES missed in dataset: ', diff, '\n')

# Проверка наличия всех регионов по каждой дате, и всех дат по каждому региону с помощью функций check_regions и check_dates
def check_region_date(data):
    
    df = data[['ReportDate','subj']]
    
    wrong_regions = ["Belarus'", 'Ukraina', 'Kazahstan']
    regions_true = pd.read_csv('data/subj.txt', encoding='utf-8', header=0, sep='|')
    regions_true = regions_true.rename(columns={regions_true.columns[0]:'subj'})
    regions_true.subj = regions_true.subj.map(lambda x: translit(x, reversed=True) if x != 'UNKNOWN'  else x)
    
    # Для каждой даты в данных проверяем наличие всех регионов
    for date in df.ReportDate.unique():
        check_regions(date, regions_true.subj.values, df[df.ReportDate == date].subj.values, wrong_regions)
    # Для каждого региона в данных проверяем наличие всех дат
    for region in df.subj.unique():
        check_dates(region, dates_true, df[df.subj == region].ReportDate)

In [84]:
# Печатает предупреждения в случае пропусков регионов или дат
check_region_date(data)

## Сверка текущих данных с выложенными ранее (с предыдущей папкой site)

In [85]:
def read_data_from_js(file_path):
    '''
    Метод считывает построчно .js файл, записывает все в большую строку flat_string, затем сохраняет в датафрейм
    param:
        file_path: Путь к файлу
    return:
        data: DataFrame с данными
    '''
    with open(file_path, 'r') as js_file:
        FIRST_LINE = True
        flat_string = ''
        # считываем построчно .js файл
        for line in js_file:
            
            flat_string += line
            break
            '''if ']' in line:
                break
            
            if FIRST_LINE == True:
                FIRST_LINE = False
                #добавляем скобку для корректного считывания json файла
                flat_string = flat_string + '['
            else:
                flat_string = flat_string + line
        #добавляем скобку для корректного считывания json файла 
        flat_string = flat_string + ']'''
        #flat_string = flat_string[8:-2]
    #print flat_string
    # Конвертим json в датафрейм
    result = pd.read_json(flat_string, convert_dates = False)
    # конвертим дату из unix timestamp в pandas timestamp
    result.date = pd.to_datetime(result.date, unit = 'ms').map(lambda x: x.replace(day=15, hour=0))
    # транслит названий регионов для сравнения
    result.region = result.region.map(lambda x: translit(x, reversed=True) if x != 'UNKNOWN'  else x)
    result.value = result.value.astype('float')
    # Переименовываем колонки к формату файла "_check.txt"
    new_column_names = ['ReportDate', 'subj', 'main_ind']
    result.rename(columns = dict(zip(result.columns,  new_column_names)), inplace = True)
    
    return result

def compare_data_with_old_js(data, old_data):
    '''
    Метод мерджит 2 таблицы и вычисляет разницу в значениях main_ind - main_ind_old, в т.ч. в %
    param:
        data: Датафрейм с новыми данными, всего 3 колонки: ReportDate, subj, main_ind
        old_data: Датафрейм со старыми данными, всего 3 колонки: ReportDate, subj, main_ind
    return:
        data_merge: Датафрейм с разницой значений в абсолюте и в %
    '''
    data_merge = old_data.merge(data, how='left', on=['subj','ReportDate'], sort=False, suffixes = ['_old',''])
    data_merge['main_ind_diff'] = data_merge.main_ind - data_merge.main_ind_old
    data_merge['main_ind_diff_percent'] = (data_merge.main_ind - data_merge.main_ind_old)/data_merge.main_ind_old

    return data_merge

In [88]:
PATH_TO_SITE_PREVIOUS = PATH_TO_SITE_DIRECTORY + 'Site_previous/'
PATH_TO_JS_FILE = PATH_TO_SITE_PREVIOUS + 'data'+ PARAMETER_NUMBER + '.js'

# считываем данные из .js в датафрейм
old_data = read_data_from_js(PATH_TO_JS_FILE)
# сравниваем датафреймы
data_compare = compare_data_with_old_js(data[['ReportDate','subj','main_ind']], old_data)
data_compare[np.abs(data_compare.main_ind_diff_percent) > 0.05]

,ReportDate,subj,main_ind_old,main_ind,main_ind_diff,main_ind_diff_percent


In [89]:
data_compare[(np.abs(data_compare.main_ind_diff_percent) > 0.02)&(data_compare.ReportDate != '2018-05-15')]

,ReportDate,subj,main_ind_old,main_ind,main_ind_diff,main_ind_diff_percent


## Медианное отклонение от медианы регионов

Находим отклонение региона по каждой дате от медианы всех остальных регионов, т.е. отклонение от общих, групповых трендов

In [90]:
#приводим даты к концу месяца
data.ReportDate = data.ReportDate.apply(lambda x: x + MonthEnd())

In [91]:
def median_absolute_deviation(data):
    """
    Метод вычисляет медианное отклонение от медианы (MAD) для данных
    """
    const = 0.6745 #0.75 percentile of normal distribution
    if data.ndim > 1 :
        MAD_ = np.median(np.abs(data - np.median(data, -1).reshape(data.shape[0], -1)), -1)/const

    if data.ndim == 1:
        MAD_ = np.median(np.abs(data - np.median(data)))/const
    
    # single dimention data
    #median_ = np.median(data, axis=0)
    #MAD_ = np.median((np.abs(data - median_)))/const
    
    return MAD_

#####################################################
def mad_region_group(data, threshold = 3.):
    """
    В проверке находится медиана регионов по каждой дате и
    отклонение от этой медианы по каждой дате.
    
    params:
        threshold: порог по которому проставляется флаг выброса, по умолчанию 3
    """
    # Находим медиану по регионам, выбрасываем Россию и неизвестные при рассчетах
    data_all = data[~data.subj.isin(['Rossija','UNKNOWN','Ne opredeleno'])]\
        .groupby(['ReportDate'], as_index=False)\
        .agg({'main_ind':'median'})\
        .rename(columns={'main_ind':'main_ind_median'})
    
    # Собираем плоскую таблицу в таблицу по датам (регионы - столбцы, даты - строки)
    for subj_ in data[~data.subj.isin(['UNKNOWN','Rossija', 'Ne opredeleno'])].subj.unique():
        #print(subj_)
        temp = data[data.subj == subj_][['ReportDate', 'main_ind']].rename(columns={'main_ind':'main_ind'+'_'+subj_}).copy()
        data_all = data_all.merge(temp, how='left', on=['ReportDate'])
    
    # Заполним пропуски в данных нулями
    data_all = data_all.fillna(0.)
    
    # Мастштабируем (стандартизируем) каждый регион в отдельности
    columns_to_scale = [col for col in data_all.columns if 'main_ind' in col]
    data_all_scaled = pd.DataFrame(StandardScaler().fit_transform(data_all[columns_to_scale]), columns=columns_to_scale)
    data_all_scaled['ReportDate'] = data_all['ReportDate'].copy()
    
    
    # Применяем MAD регионов по датам
    # Найдем медиану отмасштабированных рядов 
    columns_to_med =[col for col in data_all_scaled.columns if 'main_ind' in col 
                      and 'main_ind_median' not in col
                      and 'main_ind_Rossija' not in col]
    data_all_scaled['main_ind_median_scaled'] = data_all_scaled[columns_to_med].apply(np.median, axis=1)
    
    # Вычитаем из каждого региона медиану 
    columns_to_subtract = [col for col in data_all_scaled.columns if 'main_ind' in col 
                       and 'main_ind_median' not in col
                       and 'main_ind_median_scaled' not in col]
    for col in columns_to_subtract:
        data_all_scaled[col] = data_all_scaled[col] - data_all_scaled['main_ind_median_scaled']
    
    # Находим MAD и медиану отклонения
    data_all_scaled['MAD'] = data_all_scaled[columns_to_subtract].apply(lambda x: median_absolute_deviation(x), axis=1)
    data_all_scaled['diff_median'] = data_all_scaled[columns_to_subtract].apply(lambda x: np.median(x), axis=1)
    
    # Рассчитываем метки для точек: больше 3 медианных отклонений - выброс
    for col in columns_to_subtract:
        data_all_scaled[col + '_label'] = (np.abs(data_all_scaled[col] - data_all_scaled['diff_median']) > threshold * data_all_scaled['MAD']).astype(int)
        
    return data_all, data_all_scaled

#####################################################

def print_mad_region_group(data_all, data_all_scaled, parameter_number = PARAMETER_NUMBER):
    """
    Метод выводит в файл графики по регионам где есть отклонения
    """
    # Задаем название файла
    report_name = pd.to_datetime('now').strftime("%Y_%m_%d_%H_%M")
    pp = PdfPages('mad_region_' + parameter_number + '_' + report_name + '.pdf')
    # Поля для использования
    columns_to_use = [col for col in data_all_scaled.columns if 'main_ind' in col 
                     and 'main_ind_median' not in col
                     and 'main_ind_median_scaled' not in col
                     and '_label' not in col]
    
    for col in columns_to_use:
        # Находим метки выбросов
        list_of_inds = data_all_scaled[data_all_scaled[col + '_label'] == 1].index
        if len(list_of_inds >0):
            # Создаем график по каждому региону с выбросом
            fig, ax = plt.subplots(figsize=(20,8))
            #plt.clf()
            data_all.plot(kind='line', x='ReportDate', y=[col,'main_ind_median'], ax=ax, title=col)
            # Помечаем точки как выбросы
            for ind in list_of_inds:
                plt.annotate('outlier', xy=([data_all.ReportDate.iloc[ind],data_all[col].iloc[ind]]), xycoords='data',xytext=(+20, +20), textcoords='offset points', fontsize=14,
                             arrowprops = dict(arrowstyle="-|>", connectionstyle="arc3, rad=0.", color='green'), color='green')
            pp.savefig()
    pp.close()

In [92]:
%%time
data_all, data_all_scaled = mad_region_group(data)

CPU times: user 1.46 s, sys: 8.76 ms, total: 1.47 s
Wall time: 2.31 s


In [ ]:
print_mad_region_group(data_all, data_all_scaled, parameter_number = PARAMETER_NUMBER)

## Ансамблирование Window MAD отдельно по каждому региону

In [94]:
from itertools import product

def rolling_window(data, window):
    '''
    One of the fastest way for doing rolling windows over numpy array.
    '''
    shape = data.shape[:-1] + (data.shape[-1] - window + 1, window)
    strides = data.strides + (data.strides[-1],)
    
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

############################################

def calculate_window_mad(data, window_size = 3, threshold = 3.):
    '''
    Метод вычисляет для каждой точки из ряда data (кроме первых window_size точек)
    выброс она или нет по предыдущим window_size точкам
    params:
        data: numpy array, float, ряд 
        window_size: int, размер окна
        threshold: float, порог отсечения для метки выброс/нет
    return:
        labels_: list, int, массив меток выброс/нет
    '''
    labels_ = [0]*window_size
    # Состовляем скользящие окна для каждой точки из предыдущих точек 
    window_data = rolling_window(data, window_size)[:-1]
    # Рассчитываем медиану и MAD для каждого окна
    window_median = np.median(window_data, axis=1)
    window_mad = median_absolute_deviation(window_data)
    
    # Рассчитываем метки выброс/нет по ранее рассчитаным медианам и отклонениям
    labels_.extend((np.abs(data[window_size:] - window_median) > threshold * window_mad).astype(int))
    
    return labels_

############################################

def ensemble_window_mad(data, ensemble_threshold = 0.7, params = {'threshold': [2.5, 3., 3.5], 'window_size': [5, 7, 9]}):
    '''
    Метод ансамблирует несклько вычислений calculate_window_mad
    params:
        data: numpy array, float, ряд 
        ensemble_threshold: int, порог для выставлении метки при ансамблировании
        params: dict, зачения для threshold, window_size
    return:
        result_labels: list, метки после ансамблирования
    '''
    ensemble_labels = []
    # Для каждого значения параметров рассчитаем window MAD
    for threshold, window_size in product(params['threshold'], params['window_size']):
        labels = calculate_window_mad(data, window_size, threshold)
        ensemble_labels.append(labels)
    # Получаем по каждой точке среднее значение меток, 
    # выставляем итоговую метку по порогу ансамблирования
    result_labels_ = np.mean(ensemble_labels, axis = 0)
    result_labels = list(map(lambda x: 0 if x < ensemble_threshold else 1, result_labels_))
    
    return result_labels

In [95]:
def calculate_ensemble_window_mad_data(data, **args):
    '''
    Метод вычисляет для каждого регионального ряда метки через ансамблирование window_mad
    params:
        data: numpy array, float, ряд
        args: аргументы для ensemble_window_mad:
                ensemble_threshold: int, порог для выставлении метки при ансамблировании, 
                params: dict, зачения для threshold, window_size
    return:
        data_all: pandas dataframe, региональные ряды по столбцам + столбцы с метками
    '''
    
    # Находим медиану по регионам, выбрасываем Россию и неизвестные при рассчетах
    data_all = data[~data.subj.isin(['Rossija','UNKNOWN','Ne opredeleno'])]\
        .groupby(['ReportDate'], as_index=False)\
        .agg({'main_ind':'median'})\
        .rename(columns={'main_ind':'main_ind_median'})
    
    # Собираем плоскую таблицу в таблицу по датам (регионы - столбцы, даты - строки)
    for subj_ in data[~data.subj.isin(['UNKNOWN','Ne opredeleno'])].subj.unique():
        temp = data[data.subj == subj_][['ReportDate', 'main_ind']].rename(columns={'main_ind':'main_ind'+'_'+subj_}).copy()
        data_all = data_all.merge(temp, how='left', on=['ReportDate'])

    # для каждого регионального ряда вычисляем метки ансамбля
    col_to_use = [col for col in data_all.columns if 'main_ind' in col and 'main_ind_median' not in col]
    for col in col_to_use:
        data_all[col + '_label'] = ensemble_window_mad(data_all[col].values, **args)
    
    return data_all

############################################

def print_ensemble_window_mad(data_all, parameter_number = PARAMETER_NUMBER):
    """
    Метод выводит в файл графики по регионам где есть отклонения
    """
    # Задаем название файла
    report_name = pd.to_datetime('now').strftime("%Y_%m_%d_%H_%M")
    pp = PdfPages('ensemble_window_mad_' + parameter_number + '_' + report_name + '.pdf')
    # Поля для использования
    columns_to_use = [col for col in data_all.columns if 'main_ind' in col 
                     and 'main_ind_median' not in col
                     and '_label' not in col]
    
    for col in columns_to_use:
        # Находим метки выбросов
        list_of_inds = data_all[data_all[col + '_label'] == 1].index
        if len(list_of_inds >0):
            # Создаем график по каждому региону с выбросом
            fig, ax = plt.subplots(figsize=(20,8))
            data_all.plot(kind='line', x='ReportDate', y=[col,'main_ind_median'], ax=ax, title=col)
            # Помечаем точки как выбросы
            for ind in list_of_inds:
                plt.annotate('outlier', xy=([data_all.ReportDate.iloc[ind], data_all[col].iloc[ind]]), xycoords='data',xytext=(+20, +20), textcoords='offset points', fontsize=14,
                             arrowprops = dict(arrowstyle="-|>", connectionstyle="arc3, rad=0.", color='green'), color='green')
            pp.savefig()
    pp.close()

In [96]:
%%time
data_all_2 = calculate_ensemble_window_mad_data(data, ensemble_threshold = 0.8)

CPU times: user 1.81 s, sys: 3.95 ms, total: 1.82 s
Wall time: 1.95 s


In [ ]:
print_ensemble_window_mad(data_all_2, parameter_number = PARAMETER_NUMBER)